# Set of tools and function to use Matlab MEOP scripts within python

In [2]:
# init matlab
import matlab.engine
import io

In [3]:
from pathlib import Path
import sys
import os
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import gsw
from IPython.display import display,Image

path_MEOP_analyses = os.fspath(Path.home() / 'MEOP_analyses/')
if not (path_MEOP_analyses in sys.path):
        sys.path.insert(0,path_MEOP_analyses)

import importlib
import meop
importlib.reload(meop)

<module 'meop' from '/home/jupyter-froqu/MEOP_analyses/meop.py'>

In [4]:
def start_matlab(eng=[]):
    # Start matlab iif not already started
    try:
        eng.eval("disp('matlab already started')",nargout=0)
        print('matlab already started')
        print('PWD:',eng.pwd())
    except:
        eng = matlab.engine.start_matlab()
        print('matlab started')
        print('PWD:',eng.pwd())
    return eng

def print_matlab(namevar):
    with io.StringIO() as out:
        eng.eval(namevar,nargout=0,stdout=out,stderr=out)
        print(out.getvalue())
    
def run_command(cmd,verbose=True):
    # execute a matlab command
    with io.StringIO() as out, io.StringIO() as err:
        try:
            print(cmd)
            eng.eval(cmd,nargout=0,stdout=out,stderr=err)
            if verbose:
                print(out.getvalue())
            return True
        except:
            if verbose:
                print(err.getvalue())
            return False

In [5]:
# init mirounga and load conf
def init_mirounga():
    eng.addpath(str(processdir))
    conf = eng.eval("init_config();",nargout=1)
    run_command("conf = init_mirounga;")
    return conf

In [6]:
def load_info_deployment(EXP='',one_smru_name=''):
    init_mirounga()
    eng.workspace['EXP'] = EXP
    eng.workspace['one_smru_name'] = one_smru_name
    eng.eval("info_deployment=load_info_deployment(conf,EXP,one_smru_name);",nargout=0)

In [7]:
def process_single_deployment(EXP=''):
    load_info_deployment(EXP=EXP)
    if eng.eval("isfield(info_deployment,'invalid_code')") and eng.eval("info_deployment.invalid_code"):
        return False
    if not run_command("remove_deployment(conf,EXP);"):
        return False
    if not run_command("create_ncargo(conf,EXP);"):
        return False
    if not run_command("create_fr0(conf,EXP);"):
        return False
    if not run_command("update_metadata(conf,EXP);"):
        return False
    if not run_command("apply_adjustments(conf,EXP);"):
        return False
    if not run_command("apply_tlc(conf,EXP);"):
        return False
    if not run_command("apply_tlc_fr(conf,EXP);"):
        return False
    if not run_command("create_hr2(conf,EXP);"):
        return False
    return True

In [8]:
def process_single_tag(one_smru_name=''):
    load_info_deployment(one_smru_name=one_smru_name)
    if eng.eval("isfield(info_deployment,'invalid_code')") and eng.eval("info_deployment.invalid_code"):
        return False
    if not run_command("create_ncargo(conf,EXP,one_smru_name);"):
        return False
    if not run_command("create_fr0(conf,EXP,one_smru_name);"):
        return False
    if not run_command("update_metadata(conf,EXP,one_smru_name);"):
        return False
    if not run_command("apply_adjustments(conf,EXP,one_smru_name);"):
        return False
    if not run_command("apply_tlc(conf,EXP,one_smru_name);"):
        return False
    if not run_command("apply_tlc_fr(conf,EXP,one_smru_name);"):
        return False
    if not run_command("create_hr2(conf,EXP,one_smru_name);"):
        return False
    return True

In [9]:
# function which updates metadata based on info_meta.csv file
def update_metadata(EXP='',one_smru_name=''):

    import netCDF4

    df_meta = pd.read_csv(meop.processdir / 'table_meta.csv').set_index('smru_platform_code')

    if one_smru_name in df_meta.index:
        df_meta = df_meta.loc[[one_smru_name],:]
    elif EXP:
        df_meta['EXP'] = df_meta.index
        df_meta['EXP'] = df_meta.EXP.str.split('-').apply(lambda x: x[0])
        df_meta = df_meta.loc[df_meta.EXP == EXP,:]

    modes = ['lr0','lr1','hr0','hr1','fr0','fr1']
    for tag in df_meta.index:

        meta_row = df_meta.loc[tag,:].dropna()

        for qf in modes:

            name_prof = meop.fname_prof(tag,qf=qf)        
            if Path(name_prof).exists():

                with netCDF4.Dataset(name_prof,'a') as f:
                    for col in meta_row.keys():
                        if f.location != meta_row[col]:
                            f.location = meta_row[col]



# Code to process data

In [10]:
try:
    eng = start_matlab(eng)
except:
    eng = start_matlab()
conf = init_mirounga()

matlab started
PWD: /home/jupyter-froqu/MEOP_analyses


NameError: name 'processdir' is not defined

In [9]:
EXP = 'ct107'
load_info_deployment(EXP=EXP)
print_matlab("info_deployment")
process_single_deployment(EXP)
run_command("generate_plot1(conf,EXP);")
run_command("generate_plot2(conf,EXP);")

In [34]:
one_smru_name='ct107-933-13'
process_single_tag(one_smru_name=one_smru_name)
run_command("generate_plot1(conf,EXP,one_smru_name);")
run_command("generate_plot2(conf,EXP,one_smru_name);")

conf = init_mirounga;

create_ncargo(conf,EXP,one_smru_name);
Process smru_name=ct107-933-13
	24 tags
	4902 profiles
  ct107-933-13: 0 profiles and 0 Sprofiles removed
SMRU ct107-933-13, PTT   133774: use crawl locations

create_fr0(conf,EXP,one_smru_name);
Process hr smru_name=ct107-933-13
  SMRU ct107-933-13, PTT   133774: use crawl locations for fr data
  Date issue in ct107-933-13 hr data, 12933 instr_id: 1 profiles removed
  ct107-933-13: hr tags with 475 profiles
  ct107-933-13: 0 profiles and 0 Sprofiles removed

update_metadata(conf,EXP,one_smru_name);

apply_adjustments(conf,EXP,one_smru_name);

apply_tlc(conf,EXP,one_smru_name);

apply_tlc_fr(conf,EXP,one_smru_name);

create_hr2(conf,EXP,one_smru_name);
Create hr2 version [by default, hr1 --> hr2]
  ct107-933-13: high resolution fr1 --> hr2

generate_plot1(conf,EXP,one_smru_name);
calibration plots: ct107-933-13

generate_plot2(conf,EXP,one_smru_name);
plot diag ct107: lr0, raw
plot diag ct107: lr1, adj
plot diag ct107: hr1, 

True

In [245]:
eng.quit()